---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [ ]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [ ]:
def degree_distribution(G, only_len=True):
    deg=G.degree()
    deg_values=sorted(set(deg.values()))
    histogram=[list(deg.values()).count(i)/float(nx.number_of_nodes(G)) for i in deg_values]
    if only_len:
        return len(histogram)
    else:
        return histogram

In [ ]:
def graph_identification():
    
    graph_dict={}
    for i in range(len(P1_Graphs)):
        graph_dict['Graph_'+str(i+1)]={'avg_clustering':nx.average_clustering(P1_Graphs[i]),
                                      'avg_path':nx.average_shortest_path_length(P1_Graphs[i])}
    df=pd.DataFrame().from_dict(graph_dict, orient='index')
    df['nodes_dist']=[degree_distribution(g) for g in P1_Graphs ]
    ans=[]
    for i in range(len(df)):
        if df['nodes_dist'][i]>10:
            ans.append('PA')
        elif df['avg_clustering'][i]> 0.1:
            ans.append('SW_L')
        else:
            ans.append('SW_H')
    
    return ans

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [ ]:
from sklearn.ensemble import AdaBoostClassifier as ADA
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

def salary_predictions():
    
    df=pd.DataFrame(index=G.nodes())
    df['salary']=pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    df['department']=pd.Series(nx.get_node_attributes(G, 'Department'))
    df['clustering']=pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness centrality']=pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness centrality']=[i for i in (nx.betweenness_centrality(G, normalized=True, endpoints=False)).values()]
    df['PR']=[ i for i in nx.pagerank(G).values()]

    features = list(df.columns[1:].values)

    df_c=df.copy()
    df_c.dropna(axis=0, inplace=True)
    X_train = df_c[features]
    y_train = df_c['salary']

    #this is for y_test
    df_nan = df[df.isnull().any(axis=1)]
    X_test = df_nan[features]
    y_test = df_nan['salary']
    
    grid_values={'learning_rate': [0.4,0.5,0.6,0.7,0.8],
                'n_estimators': [25,35,45,50,55]}
    clf=ADA()
    grid_clf_auc=GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
    grid_clf_auc.fit(X_train, y_train)
    precit_y = grid_clf_auc.predict_proba(X_test)[:,1]
    
    print(f'best params are: {grid_clf_auc.best_params_}')
    print(f'best score: {grid_clf_auc.best_score_}')
    
    return pd.Series(precit_y, X_test.index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [4]:
from sklearn.ensemble import AdaBoostClassifier as ADA

def new_connections_predictions():
    for node in G.nodes():
        G.node[node]['community']=G.node[node]['Department']

    df=future_connections.copy()
    df['preferential_attch'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['common neighbors'] = df.index.map(lambda edge: len(list(nx.common_neighbors(G,edge[0],edge[1]))))
    df['jaccard coef']=[i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['res_alloc_index']=[i[2] for i in nx.resource_allocation_index(G, df.index)]
    df['AA-index']=[i[2] for i in nx.adamic_adar_index(G, df.index)]
    df['community neighbors']=[i[2] for i in nx.cn_soundarajan_hopcroft(G, df.index)]
    df['communuity res_alloc']=[ i[2] for i in nx.ra_index_soundarajan_hopcroft(G, df.index)]

    features = list(df.columns[1:].values)

    df_c=df.copy()
    df_c.dropna(axis=0, inplace=True)
    X_train = df_c[features]
    y_train = df_c['Future Connection']

    df_nan=df[df.isnull().any(axis=1)]
    X_test=df_nan[features]


    grid_values={'learning_rate': [0.4,0.5,0.6,0.7,0.8,1.0],
                'n_estimators': [25,35,45,50,55,65]}
    clf2=ADA()
    grid_clf2_auc=GridSearchCV(clf2, param_grid=grid_values, scoring='roc_auc')
    grid_clf2_auc.fit(X_train, y_train)
    predict_y2 = grid_clf2_auc.predict_proba(X_test)[:,1]
    results= pd.Series(predict_y2, X_test.index)

    print(f'best params are: {grid_clf2_auc.best_params_}')
    print(f'best score: {grid_clf2_auc.best_score_}')
    

    return results

In [5]:
new_connections_predictions()

NameError: name 'GridSearchCV' is not defined